In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60

Autosaving every 60 seconds


In [9]:
from ivanocode.pipeline.pipeline import *

DEBUG [2020-05-31 16:08:18] root: Device: cpu


Parsing args


 INFO [2020-05-31 16:08:20] root: Not running pipeline


Patching up DataFrame.copy


In [ ]:
reproducibility_mode()
sales_series = read_series_sample(n_total_series)
sales_series = melt_sales_series(sales_series)
sales_series = extract_day_ids(sales_series)
sales_series = join_w_calendar(sales_series)
sales_series = join_w_prices(sales_series)

In [19]:
encoders = {}

In [26]:
sales_series['parquet_partition'] = np.random.randint(1, 300, sales_series.shape[0]).astype('uint8')
from sklearn.preprocessing import LabelEncoder
if 'day_date' in sales_series.columns:
    sales_series.drop(['day_date'], axis=1, inplace=True)

for col in sales_series.columns:
    if col in encoders:
        print(f'{col} already encoded - skipping')
        continue

    # petastorm can't read these
    if str(sales_series[col].dtype) == 'uint8':
        sales_series[col] = sales_series[col].astype('int')

    if str(sales_series[col].dtype) in ['category', 'object']:
        print(f'{col} - encoding')
        enc = LabelEncoder()
        sales_series[col] = enc.fit_transform(sales_series[col])
        encoders[col] = enc

sales_series.info()

id already encoded - skipping
item_id already encoded - skipping
dept_id already encoded - skipping
cat_id already encoded - skipping
store_id already encoded - skipping
state_id already encoded - skipping
day_date_str already encoded - skipping
date already encoded - skipping
<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 16 columns):
id                   int64
item_id              int64
dept_id              int64
cat_id               int64
store_id             int64
state_id             int64
day_id               int16
sales                int16
day_date_str         int64
month_id             int64
date                 int64
wm_yr_wk             int16
snap_flag            int64
sell_price           float32
sales_dollars        float32
parquet_partition    int64
dtypes: float32(2), int16(3), int64(11)
memory usage: 6.0 GB


In [28]:
for name, enc in encoders.items():
    np.save(f'{name}.npy', enc.classes_)

In [29]:
# TODO: uint -> int, category/object -> int, day_date -> drop
sales_series.to_parquet(
    'sales_series_melt.parquet',
    index=False,
    partition_cols=['parquet_partition'],
    row_group_size=1000
)